### Importações

In [1]:
# Package                      Version
# ---------------------------- --------
# matplotlib                   3.7.2
# opencv-python                4.8.0.74
# tensorflow                   2.10.1
# tensorflow-gpu               2.10.1
import cv2
import numpy as np
import os
import random
import uuid
from matplotlib import pyplot as plt
# Importando dependencias de TensorFlow
import tensorflow as tf
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten

# embeddings Base para redes neurais convolucionais com TensorFlow:
# Model(input=[inputImage, inputVerification], output=[1,0])
# Input(shape=(100,100,3))

# Configurando uso de GPU e limites de memória
# https://www.tensorflow.org/guide/gpu
# https://www.tensorflow.org/guide/gpu#limiting_gpu_memory_growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        try:
        # Habilita uso de memória da GPU
            tf.config.experimental.set_memory_growth(gpu, True)
            print('GPU: ', gpu)
        # Limita uso de memória da GPU
        #    tf.config.experimental.set_virtual_device_configuration(gpu, [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
        except RuntimeError as e:
            print(e)
# Saída aguardada conforme GPU disponível:"""" GPU:  PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')""""
else:
    print('Não foi localizado nenhuma GPU')

GPU:  PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


### Pegando caminhos de pastas Imagens
* Referencia    =>    (anchor)
* Cadastro      =>    (positive)
* Aleatórias    =>    (negative)

In [2]:
# Verificando se as pasta (Positivo, Negativo e Ancora ) existem, senão criaremos
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

#  ID de teste para o usuário ( RA )
userID = 1001

# Capturar Imagens de Ancora
POS_PATH_ID = os.path.join('data', 'positive',str(userID))
if not os.path.exists(POS_PATH_ID):
    os.makedirs(POS_PATH_ID)

ANC_PATH_ID = os.path.join('data', 'anchor', str(userID))
if not os.path.exists(ANC_PATH_ID):
    os.makedirs(ANC_PATH_ID)

# print(NEG_PATH)
# print(ANC_PATH_ID)
# print(POS_PATH_ID)

| correndo imagens

In [3]:
# embeddings Base para redes neurais convolucionais com TensorFlow:
anchor = tf.data.Dataset.list_files(ANC_PATH_ID +'\*.jpg').take(300)
positive = tf.data.Dataset.list_files(POS_PATH_ID +'\*.jpg').take(300)
negative = tf.data.Dataset.list_files(NEG_PATH +'\*.jpg').take(300)

# # TEST: Imprimindo os caminhos das imagens
# dir_test = anchor.as_numpy_iterator()
# for i in range(10):
#     print(dir_test.next())

### Função de Pré Processamento

In [4]:
def preprocess(file_path):

    # Ler o arquivo de imagem
    byte_img = tf.io.read_file(file_path)

    # Decodificar o arquivo de imagem com TensorFlow
    img = tf.image.decode_jpeg(byte_img)

    #  Redimensionar a imagem [100x100x3]
    img = tf.image.resize(img, [100, 100])
    img = img / 255.0
    
    return img

# img = preprocess('data\\anchor\\1001\\c7d4d103-2496-11ee-ad27-00e04f0d8f2c.jpg')

# print(f'Min: {img.numpy().min()} - Max:{img.numpy().max()}')

# TEST: Imprimindo as imagens
# plt.imshow(img)

# tf.ones_like([1,1,1,1,1,444,4,43,2])
# tf.zeros_like([0,1,0,3])
# exemplos tf 
# (imfBase , imgPositiva) => 1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
# (imgBase, imgNegativa) => 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

# tf.ones(len(anchor))
# tf.zeros(len(anchor))

# class_labels = tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))
# iterador = class_labels.as_numpy_iterator()
# iterador.next()

In [5]:
positives = tf.data.Dataset.zip((anchor , positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor , negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)
# samples do resultado do dataset
# amostras = data.as_numpy_iterator()
# # amostras.next()
# exemplo = amostras.next()
# exemplo

### Função de Pré Processamento de imagens idênticas ou distintas

In [6]:
# Função de verificação para 'gemeos', ou seja, se as são a mesma pessoa 
def preprocess_twin(input_img, validation_img, label):
    return (preprocess(input_img), preprocess(validation_img), label)
#  o parametro '*' em '*exemplo' é para desempacotar a tupla
# resultado = preprocess_twin(*exemplo)
# plt.imshow(resultado[0])

In [7]:
#  Build Datapipe para treinamento
#  https://www.tensorflow.org/guide/data_performance
#  https://www.tensorflow.org/guide/data

data = data.map(preprocess_twin)
data.cache()
data.shuffle(buffer_size=1024)

# Demonstrando separação de dados para treino e teste 1.0 para mesma pessoas  e 0.0 para pessoas diferentes (anchor , (positiva ou negativa), label)   ))
# amostra2 = data.as_numpy_iterator()
# len(amostra2.next())
# exemplar = amostra2.next()
# for i in range(10):
#     exemplar = amostra2.next()
#     print(exemplar[2])

<ShuffleDataset element_spec=(TensorSpec(shape=(100, 100, None), dtype=tf.float32, name=None), TensorSpec(shape=(100, 100, None), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None))>

In [8]:
# Training partition
# Podendo utilizar um percentual das imagens com 'round(len(data) * 0.5)' ou 'take(400)' para 400 imagens
# print(round(len(data) * 0.5))
train_data = data.take(round(len(data) * 0.7))
# Criaremos lote com '16' imagens
train_data = train_data.batch(16)
# Creiaremos um buffer de 8 imagens para pré-carregamento
train_data = train_data.prefetch(8)

# EXEMPLO: Imprimindo o resultado do treino
# train_samplas = train_data.as_numpy_iterator()
# train_sampla = train_samplas.next()
# print(train_sampla[0].shape)
# Saída: (16, 100, 100, 3)

In [9]:
def make_embedding():
    inp = Input(shape=(100, 100, 3), name='input_image')
    #  Primeiro Bloco
    c1 = Conv2D(64, (10, 10), activation='relu')(inp)
    m1 = MaxPooling2D(64, (2, 2), padding='same')(c1)
    
    # Segundo Bloco
    c2 = Conv2D(128, (7, 7), activation='relu')(m1)
    m2 = MaxPooling2D(64, (2, 2), padding='same')(c2)
    
    # Terceiro Bloco
    c3 = Conv2D(128, (4, 4), activation='relu')(m2)
    m3 = MaxPooling2D(64, (2, 2), padding='same')(c3)
    
    # Bloco Final
    c4 = Conv2D(256, (4, 4), activation='relu')(m3)
    fl = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid')(fl)
     
    # embedding <=> incorporação
    return Model(inputs=[inp], outputs=[d1], name='embedding')

embedding = make_embedding()

# embedding.summary()

In [10]:
# Classe de Layer para calcular a distância entre as imagens
class L1Dist(Layer):
    def __init__(self, **kwargs):
        super().__init__()
    
    def call (self, input_embedding, validation_embedding):
        # Calcula a distância entre as imagens
        return tf.math.abs(input_embedding - validation_embedding)

In [11]:
def make_siamese_model():
    # Entrada para a imagem base
    inp = Input(shape=(100, 100, 3), name='input_image')
    # Entrada para a imagem de validação
    val = Input(shape=(100, 100, 3), name='validation_image')
    
    siamese_layer = L1Dist()
    siamese_layer._name = 'distance'
    distance = siamese_layer(embedding(inp), embedding(val))

    # Camanda de Classificador
    classifier = Dense(1, activation='sigmoid')(distance)

    return Model(inputs=[inp, val], outputs=[classifier], name='SiameseNetwork')

siamese_model = make_siamese_model()
# siamese_model.summary()

### TREINO

In [12]:
# Função de verificação de perda ou  Acurácia 
# binary_cross_loss = tf.losses.BinaryCrossentropy(from_logits=True)
# Isso é recomendado quando você está lidando com um modelo que produz valores de saída não normalizados, como logits.
# https://www.tensorflow.org/api_docs/python/tf/keras/losses/BinaryCrossentropy
binary_cross_loss = tf.losses.BinaryCrossentropy()

# optimizer => Otimizador para o modelo, keras.optimizers.Adam com a taxa de aprendizado padrão.
# 1e-4 => 0.0001
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam
opt = tf.keras.optimizers.Adam(1e-4)

# Estabilizando o modelo 
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=opt, siamese_model=siamese_model)


In [13]:
@tf.function
def train_step(batch):
# Gravar todas as operações em um gráfico computacional
    with tf.GradientTape() as tape:
        # Pegar anchor e positive/negatine image
        x = batch[:2]
        # Pegar label
        y = batch[2]
        #  Forward pass -  
        yhat = siamese_model(x, training=True)
        # Calcular perda
        loss = binary_cross_loss(y, yhat)
    print(loss)
    # Calcular gradiente
    gradients = tape.gradient(loss, siamese_model.trainable_variables)
    # Atualizar pesos
    opt.apply_gradients(zip(gradients, siamese_model.trainable_variables))
    
    return loss     

In [14]:
def train(data, EPOCHS):
    
    for epoch in range(EPOCHS+1):
        print('\n Epoch {}/{}'.format(epoch, EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data))
        
        for idx, batch in enumerate(data):
            train_step(batch)
            progbar.update(idx+1)

        # Salvar modelo - epoch <=> época (atual)
        if epoch % 10 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)

### Treino do Modelo

In [15]:
EPOCH = 10
train(train_data, EPOCH)



 Epoch 0/10
Tensor("binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)


: 

: 